In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [0]:
df=pd.read_csv('/content/drive/My Drive/stock_model_data_v3.0.csv')

In [0]:
df.shape

(1375085, 49)

In [0]:
df.set_index("Date",inplace=True)
df.index=pd.to_datetime(df.index)
df.index

DatetimeIndex(['2015-03-17', '2015-03-18', '2015-03-19', '2015-03-20',
               '2015-03-23', '2015-03-24', '2015-03-25', '2015-03-26',
               '2015-03-27', '2015-03-30',
               ...
               '2020-02-06', '2020-02-07', '2020-02-10', '2020-02-11',
               '2020-02-12', '2020-02-13', '2020-02-14', '2020-02-17',
               '2020-02-18', '2020-02-19'],
              dtype='datetime64[ns]', name='Date', length=1375085, freq=None)

In [0]:
df=df.iloc[:70000,:]

In [0]:
df_train=df.iloc[0:1,:]
def prepare_train(df):
    global df_train
    if(len(df)<110):
        return
    temp=df.iloc[:(len(df)-50),:]
    df_train=df_train.append(temp)
        
df.iloc[1:,:].groupby('Symbols').apply(prepare_train)   

""


In [0]:
df_train.shape

(66143, 48)

In [0]:
symbols=df_train['Symbols']
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler(feature_range=(0,1))
df_train_scaled_values=scaler.fit_transform(df_train.iloc[:,1:])

In [0]:
df_train_scaled=pd.DataFrame(df_train_scaled_values,columns=df_train.columns[1:])
df_train_scaled['Symbols']=symbols.values
df_train_scaled=df_train_scaled[df_train.columns]
df_train_scaled.head()

,Symbols,bin_gain,bin_gain_1%,bin_gain_2%,SMA_PCH5,SMA_PCH15,SMA_PCH50,EMA_PCH5,EMA_PCH15,EMA_PCH50,ROC5,ROC15,ROC50,MOM_PCH5,MOM_PCH15,MOM_PCH50,macdhist,RSI,StochK,StochD,CMO,AroonUp,AroonDown,ChaikinAD,MFI,WillR,ULTOS,MA_BIN5,MA_BIN15,MA_BIN50,EMA_BIN5,EMA_BIN15,EMA_BIN50,ROC_BIN5,ROC_BIN15,ROC_BIN50,MOM_BIN5,MOM_BIN15,MOM_BIN50,macd_BIN,Beta_DOW,Beta_NDQ,Beta_NKK,Beta_NFTY,Beta_EURINR,Beta_GBPINR,Beta_USDINR,Beta_JPYINR
0,20MICRONS.NS,0.0,0.0,0.0,0.389976,0.324101,0.315688,0.438466,0.346913,0.377649,0.288815,0.289324,0.109555,0.670016,0.731595,0.864214,0.524334,0.450929,0.266668,0.213890,0.450929,1.00,0.24,0.501681,0.313860,0.208955,0.263301,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.388508,0.333363,0.548892,0.712064,0.557186,0.308617,0.552198,0.604795
1,20MICRONS.NS,1.0,0.0,0.0,0.385249,0.321931,0.312051,0.434991,0.344444,0.375629,0.289578,0.286690,0.107360,0.670837,0.729065,0.861528,0.524370,0.431746,0.000000,0.144445,0.431746,0.96,0.20,0.501654,0.408889,0.208333,0.241850,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.379764,0.320953,0.576092,0.761647,0.541427,0.281178,0.564579,0.580121
2,20MICRONS.NS,0.0,0.0,0.0,0.389029,0.324269,0.312292,0.438550,0.347581,0.378290,0.285018,0.296460,0.107309,0.665900,0.738307,0.861464,0.524452,0.444681,0.133342,0.133336,0.444681,0.92,0.16,0.501633,0.413597,0.250001,0.232326,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.374781,0.322024,0.569329,0.755545,0.589611,0.301626,0.566294,0.596260
3,20MICRONS.NS,0.0,0.0,0.0,0.387842,0.322645,0.309843,0.437002,0.346791,0.377759,0.285762,0.300460,0.109460,0.666711,0.741981,0.864100,0.524496,0.434162,0.000000,0.044447,0.434162,0.88,0.12,0.501618,0.392057,0.217391,0.226705,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.386264,0.328467,0.570702,0.752598,0.593600,0.306828,0.575836,0.595755
4,20MICRONS.NS,1.0,0.0,0.0,0.377350,0.313414,0.301169,0.426072,0.337193,0.369519,0.277349,0.284491,0.106834,0.657406,0.726931,0.860870,0.524358,0.375908,0.000000,0.044447,0.375908,1.00,0.08,0.501581,0.394888,0.047618,0.194009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.390538,0.339344,0.546513,0.755586,0.517460,0.264823,0.587189,0.574975


In [0]:
df_train_scaled.shape

(66143, 48)

In [0]:
X_train=[]
y_train=[]
def prepare_train_arr(df):
    for i in range(50,len(df)):
        X_train.append(df.iloc[i-50:i,3:].values)
        y_train.append(df.iloc[i,3])
    
df_train_scaled.groupby(["Symbols"]).apply(prepare_train_arr)
X_train=np.array(X_train)
X_train.shape

(62393, 50, 45)

In [0]:
df_symbols=df['Symbols']
df_scaled_values=scaler.transform(df.iloc[:,1:])
df_scaled=pd.DataFrame(df_scaled_values,columns=df.columns[1:])
df_scaled['Symbols']=df_symbols.values
df_scaled=df_scaled[df.columns]
df_scaled.head()

,Symbols,bin_gain,bin_gain_1%,bin_gain_2%,SMA_PCH5,SMA_PCH15,SMA_PCH50,EMA_PCH5,EMA_PCH15,EMA_PCH50,ROC5,ROC15,ROC50,MOM_PCH5,MOM_PCH15,MOM_PCH50,macdhist,RSI,StochK,StochD,CMO,AroonUp,AroonDown,ChaikinAD,MFI,WillR,ULTOS,MA_BIN5,MA_BIN15,MA_BIN50,EMA_BIN5,EMA_BIN15,EMA_BIN50,ROC_BIN5,ROC_BIN15,ROC_BIN50,MOM_BIN5,MOM_BIN15,MOM_BIN50,macd_BIN,Beta_DOW,Beta_NDQ,Beta_NKK,Beta_NFTY,Beta_EURINR,Beta_GBPINR,Beta_USDINR,Beta_JPYINR
0,20MICRONS.NS,0.0,0.0,0.0,0.389976,0.324101,0.315688,0.438466,0.346913,0.377649,0.288815,0.289324,0.109555,0.670016,0.731595,0.864214,0.524334,0.450929,0.266668,0.213890,0.450929,1.00,0.24,0.501681,0.313860,0.208955,0.263301,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.388508,0.333363,0.548892,0.712064,0.557186,0.308617,0.552198,0.604795
1,20MICRONS.NS,1.0,0.0,0.0,0.385249,0.321931,0.312051,0.434991,0.344444,0.375629,0.289578,0.286690,0.107360,0.670837,0.729065,0.861528,0.524370,0.431746,0.000000,0.144445,0.431746,0.96,0.20,0.501654,0.408889,0.208333,0.241850,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.379764,0.320953,0.576092,0.761647,0.541427,0.281178,0.564579,0.580121
2,20MICRONS.NS,0.0,0.0,0.0,0.389029,0.324269,0.312292,0.438550,0.347581,0.378290,0.285018,0.296460,0.107309,0.665900,0.738307,0.861464,0.524452,0.444681,0.133342,0.133336,0.444681,0.92,0.16,0.501633,0.413597,0.250001,0.232326,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.374781,0.322024,0.569329,0.755545,0.589611,0.301626,0.566294,0.596260
3,20MICRONS.NS,0.0,0.0,0.0,0.387842,0.322645,0.309843,0.437002,0.346791,0.377759,0.285762,0.300460,0.109460,0.666711,0.741981,0.864100,0.524496,0.434162,0.000000,0.044447,0.434162,0.88,0.12,0.501618,0.392057,0.217391,0.226705,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.386264,0.328467,0.570702,0.752598,0.593600,0.306828,0.575836,0.595755
4,20MICRONS.NS,1.0,0.0,0.0,0.377350,0.313414,0.301169,0.426072,0.337193,0.369519,0.277349,0.284491,0.106834,0.657406,0.726931,0.860870,0.524358,0.375908,0.000000,0.044447,0.375908,1.00,0.08,0.501581,0.394888,0.047618,0.194009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.390538,0.339344,0.546513,0.755586,0.517460,0.264823,0.587189,0.574975


In [0]:
df_scaled.shape

(70000, 48)

In [0]:
X_test=[]
y_test=[]
def prepare_test(df):
    if(len(df)<110):
        return
    for i in range(len(df)-50,len(df)):
        X_test.append(df.iloc[i-50:i,3:].values)
        y_test.append(df.iloc[i,3])
df_scaled.groupby(["Symbols"]).apply(prepare_test)
X_test=np.array(X_test)
X_test.shape

(3750, 50, 45)

In [0]:
y_train=np.array(y_train)
y_test=np.array(y_test)

In [0]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras import optimizers
import keras
import tensorflow as tf

Using TensorFlow backend.


In [0]:
df['bin_gain_2%'].value_counts()

0    58121
1    11879
Name: bin_gain_2%, dtype: int64

In [0]:
model=Sequential()
model.add(LSTM(units=50,return_sequences=True,input_shape=(X_train.shape[1],45)))
model.add(Dropout(0.2))

model.add(LSTM(units=50,return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=50,return_sequences=True))
model.add(Dropout(0.2))


model.add(LSTM(units=50))
model.add(Dropout(0.2))


model.add(Dense(units=1,activation='sigmoid'))

class_weight={0:0.10,
               1:0.90}
model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['binary_accuracy'])

In [0]:
model.fit(X_train, y_train, batch_size=500, nb_epoch=100,class_weight=class_weight)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Epoch 1/100
62393/62393 [==============================] - 62s 989us/step - loss: 0.1528 - binary_accuracy: 0.1724
Epoch 2/100
62393/62393 [==============================] - 59s 943us/step - loss: 0.1520 - binary_accuracy: 0.1682
Epoch 3/100
62393/62393 [==============================] - 58s 928us/step - loss: 0.1511 - binary_accuracy: 0.1751
Epoch 4/100
62393/62393 [==============================] - 58s 928us/step - loss: 0.1505 - binary_accuracy: 0.1887
Epoch 5/100
62393/62393 [==============================] - 57s 919us/step - loss: 0.1500 - binary_accuracy: 0.2053
Epoch 6/100
62393/62393 [==============================] - 58s 925us/step - loss: 0.1494 - binary_accuracy: 0.2209
Epoch 7/100
62393/62393 [==============================] - 58s 928us/step - loss: 0.1493 - binary_accuracy: 0.2240
Epoch 8/100
62393/62393 [==============================] - 58s 930us/step - loss: 0.1491 - binary_accuracy: 0.2218
Epoch 9/100
62393/62393 [==============================] - 57s 912us/step - loss

In [0]:
from sklearn.metrics import precision_score,accuracy_score
y_predicted=model.predict_classes(X_test)
y_predicted=y_predicted.reshape(1,-1)[0]
precision = precision_score(y_test, y_predicted)
accuracy=accuracy_score(y_test,y_predicted)
print("Test Precision is ",precision)
print("Test Accuracy is ",accuracy)

Test Precision is  0.1877818778187782
Test Accuracy is  0.4176


In [0]:
pd.crosstab(y_test, y_predicted, rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,0,1,All
True,,,
0.0,1108,1981,3089
1.0,203,458,661
All,1311,2439,3750


In [0]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_predicted))
print(pd.DataFrame(confusion_matrix(y_test, y_predicted),columns=['0','1'],index=['0','1']))

              precision    recall  f1-score   support

         0.0       0.85      0.36      0.50      3089
         1.0       0.19      0.69      0.30       661

    accuracy                           0.42      3750
   macro avg       0.52      0.53      0.40      3750
weighted avg       0.73      0.42      0.47      3750

      0     1
0  1108  1981
1   203   458


In [0]:
from sklearn.metrics import precision_score,accuracy_score
y_predicted=model.predict_classes(X_train)
y_predicted=y_predicted.reshape(1,-1)[0]
precision = precision_score(y_train, y_predicted)
accuracy=accuracy_score(y_train,y_predicted)
print("Train Precision is ",precision)
print("Train Accuracy is ",accuracy)

Train Precision is  0.25149788822316077
Train Accuracy is  0.5076531021108137


In [0]:
pd.crosstab(y_train, y_predicted, rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,0,1,All
True,,,
0.0,21432,30482,51914
1.0,237,10242,10479
All,21669,40724,62393


In [0]:
from sklearn.metrics import classification_report
print(classification_report(y_train,y_predicted))

              precision    recall  f1-score   support

         0.0       0.99      0.41      0.58     51914
         1.0       0.25      0.98      0.40     10479

    accuracy                           0.51     62393
   macro avg       0.62      0.70      0.49     62393
weighted avg       0.87      0.51      0.55     62393



In [0]:
model=Sequential()
model.add(LSTM(units=50,return_sequences=True,input_shape=(X_train.shape[1],45)))
model.add(Dropout(0.2))

model.add(LSTM(units=50,return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=50,return_sequences=True))
model.add(Dropout(0.2))


model.add(LSTM(units=50))
model.add(Dropout(0.2))


model.add(Dense(units=1,activation='sigmoid'))

class_weight={0:0.10,
               1:0.90}
optimizer=optimizers.Adam(lr=0.005)
model.compile(loss='binary_crossentropy',
                  optimizer=optimizer,
                  metrics=['binary_accuracy'])


In [0]:
model.fit(X_train, y_train, batch_size=300, nb_epoch=100,class_weight=class_weight)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Epoch 1/100
62393/62393 [==============================] - 96s 2ms/step - loss: 0.1348 - binary_accuracy: 0.4306
Epoch 2/100
62393/62393 [==============================] - 96s 2ms/step - loss: 0.1342 - binary_accuracy: 0.4358
Epoch 3/100
62393/62393 [==============================] - 96s 2ms/step - loss: 0.1347 - binary_accuracy: 0.4350
Epoch 4/100
62393/62393 [==============================] - 95s 2ms/step - loss: 0.1342 - binary_accuracy: 0.4375
Epoch 5/100
62393/62393 [==============================] - 95s 2ms/step - loss: 0.1333 - binary_accuracy: 0.4449
Epoch 6/100
62393/62393 [==============================] - 95s 2ms/step - loss: 0.1345 - binary_accuracy: 0.4396
Epoch 7/100
62393/62393 [==============================] - 96s 2ms/step - loss: 0.1343 - binary_accuracy: 0.4402
Epoch 8/100
62393/62393 [==============================] - 95s 2ms/step - loss: 0.1336 - binary_accuracy: 0.4420
Epoch 9/100
62393/62393 [==============================] - 95s 2ms/step - loss: 0.1334 - binary_

In [0]:
from sklearn.metrics import precision_score,accuracy_score
y_predicted=model.predict_classes(X_test)
y_predicted=y_predicted.reshape(1,-1)[0]
precision = precision_score(y_test, y_predicted)
accuracy=accuracy_score(y_test,y_predicted)
print("Test Precision is ",precision)
print("Test Accuracy is ",accuracy)

Test Precision is  0.18862275449101795
Test Accuracy is  0.43546666666666667


In [0]:
pd.crosstab(y_test, y_predicted, rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,0,1,All
True,,,
0.0,1192,1897,3089
1.0,220,441,661
All,1412,2338,3750


In [0]:
model=Sequential()
model.add(LSTM(units=50,return_sequences=True,input_shape=(X_train.shape[1],45)))
model.add(Dropout(0.2))

model.add(LSTM(units=50,return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=50,return_sequences=True))
model.add(Dropout(0.2))


model.add(LSTM(units=50))
model.add(Dropout(0.2))


model.add(Dense(units=1,activation='sigmoid'))

class_weight={0:0.20,
               1:0.80}
optimizer=optimizers.Adam(lr=0.005)
model.compile(loss='binary_crossentropy',
                  optimizer=optimizer,
                  metrics=['binary_accuracy'])

In [0]:
model.fit(X_train, y_train, batch_size=300, nb_epoch=100,class_weight=class_weight)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Epoch 1/100
62393/62393 [==============================] - 66s 1ms/step - loss: 0.2070 - binary_accuracy: 0.8117
Epoch 2/100
62393/62393 [==============================] - 63s 1ms/step - loss: 0.2067 - binary_accuracy: 0.8274
Epoch 3/100
62393/62393 [==============================] - 64s 1ms/step - loss: 0.2052 - binary_accuracy: 0.7756
Epoch 4/100
62393/62393 [==============================] - 63s 1ms/step - loss: 0.2036 - binary_accuracy: 0.7404
Epoch 5/100
62393/62393 [==============================] - 63s 1ms/step - loss: 0.2022 - binary_accuracy: 0.7310
Epoch 6/100
62393/62393 [==============================] - 63s 1ms/step - loss: 0.2016 - binary_accuracy: 0.7375
Epoch 7/100
62393/62393 [==============================] - 63s 1ms/step - loss: 0.2014 - binary_accuracy: 0.7404
Epoch 8/100
62393/62393 [==============================] - 62s 1ms/step - loss: 0.2011 - binary_accuracy: 0.7343
Epoch 9/100
62393/62393 [==============================] - 62s 998us/step - loss: 0.2008 - binar

In [0]:
from sklearn.metrics import precision_score,accuracy_score
y_predicted=model.predict_classes(X_train)
y_predicted=y_predicted.reshape(1,-1)[0]
precision = precision_score(y_train, y_predicted)
accuracy=accuracy_score(y_train,y_predicted)
print("Train Precision is ",precision)
print("Train Accuracy is ",accuracy)

Train Precision is  0.3713198253666182
Train Accuracy is  0.758354302565993


In [0]:
pd.crosstab(y_train, y_predicted, rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,0,1,All
True,,,
0.0,40682,11232,51914
1.0,3845,6634,10479
All,44527,17866,62393


In [0]:
from sklearn.metrics import classification_report
print(classification_report(y_train,y_predicted))

              precision    recall  f1-score   support

         0.0       0.91      0.78      0.84     51914
         1.0       0.37      0.63      0.47     10479

    accuracy                           0.76     62393
   macro avg       0.64      0.71      0.66     62393
weighted avg       0.82      0.76      0.78     62393



In [0]:
y_predicted=model.predict_classes(X_test)
y_predicted=y_predicted.reshape(1,-1)[0]
precision = precision_score(y_test, y_predicted)
accuracy=accuracy_score(y_test,y_predicted)
print("Test Precision is ",precision)
print("Test Accuracy is ",accuracy)

Test Precision is  0.21180555555555555
Test Accuracy is  0.6466666666666666


In [0]:
pd.crosstab(y_test, y_predicted, rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,0,1,All
True,,,
0.0,2181,908,3089
1.0,417,244,661
All,2598,1152,3750


In [0]:
model=Sequential()
model.add(LSTM(units=50,return_sequences=True,input_shape=(X_train.shape[1],45)))
model.add(Dropout(0.2))

model.add(LSTM(units=50,return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=50,return_sequences=True))
model.add(Dropout(0.2))


model.add(LSTM(units=50))
model.add(Dropout(0.2))


model.add(Dense(units=1,activation='sigmoid'))

class_weight={0:0.20,
               1:0.80}
optimizer=optimizers.Adam(lr=0.005)
model.compile(loss='binary_crossentropy',
                  optimizer=optimizer,
                  metrics=['binary_accuracy'])

In [0]:
model.fit(X_train, y_train, batch_size=300, nb_epoch=200,class_weight=class_weight)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Epoch 1/200
62393/62393 [==============================] - 66s 1ms/step - loss: 0.2073 - binary_accuracy: 0.8069
Epoch 2/200
62393/62393 [==============================] - 66s 1ms/step - loss: 0.2068 - binary_accuracy: 0.8301
Epoch 3/200
62393/62393 [==============================] - 65s 1ms/step - loss: 0.2069 - binary_accuracy: 0.8317
Epoch 4/200
62393/62393 [==============================] - 65s 1ms/step - loss: 0.2066 - binary_accuracy: 0.8255
Epoch 5/200
62393/62393 [==============================] - 65s 1ms/step - loss: 0.2038 - binary_accuracy: 0.7407
Epoch 6/200
62393/62393 [==============================] - 66s 1ms/step - loss: 0.2024 - binary_accuracy: 0.7391
Epoch 7/200
62393/62393 [==============================] - 67s 1ms/step - loss: 0.2019 - binary_accuracy: 0.7319
Epoch 8/200
62393/62393 [==============================] - 66s 1ms/step - loss: 0.2013 - binary_accuracy: 0.7308
Epoch 9/200
62393/62393 [==============================] - 66s 1ms/step - loss: 0.2009 - binary_

In [0]:
from sklearn.metrics import precision_score,accuracy_score
y_predicted=model.predict_classes(X_train)
y_predicted=y_predicted.reshape(1,-1)[0]
precision = precision_score(y_train, y_predicted)
accuracy=accuracy_score(y_train,y_predicted)
print("Train Precision is ",precision)
print("Train Accuracy is ",accuracy)

Train Precision is  0.3897835332345773
Train Accuracy is  0.7581138909813601


In [0]:
pd.crosstab(y_train, y_predicted, rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,0,1,All
True,,,
0.0,39144,12770,51914
1.0,2322,8157,10479
All,41466,20927,62393


In [0]:
from sklearn.metrics import classification_report
print(classification_report(y_train,y_predicted))

              precision    recall  f1-score   support

         0.0       0.94      0.75      0.84     51914
         1.0       0.39      0.78      0.52     10479

    accuracy                           0.76     62393
   macro avg       0.67      0.77      0.68     62393
weighted avg       0.85      0.76      0.78     62393



In [0]:
y_predicted=model.predict_classes(X_test)
y_predicted=y_predicted.reshape(1,-1)[0]
precision = precision_score(y_test, y_predicted)
accuracy=accuracy_score(y_test,y_predicted)
print("Test Precision is ",precision)
print("Test Accuracy is ",accuracy)

Test Precision is  0.1870342771982116
Test Accuracy is  0.5997333333333333


In [0]:
pd.crosstab(y_test, y_predicted, rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,0,1,All
True,,,
0.0,1998,1091,3089
1.0,410,251,661
All,2408,1342,3750


In [0]:
model=Sequential()
model.add(LSTM(units=50,return_sequences=True,input_shape=(X_train.shape[1],45)))
model.add(Dropout(0.2))

model.add(LSTM(units=50,return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=50,return_sequences=True))
model.add(Dropout(0.2))


model.add(LSTM(units=50))
model.add(Dropout(0.2))


model.add(Dense(units=1,activation='sigmoid'))

class_weight={0:0.10,
               1:0.90}
model.compile(loss='binary_crossentropy',
                  optimizer='adam',
              metrics=['binary_accuracy'])

In [0]:
model.fit(X_train, y_train, batch_size=500, nb_epoch=200,class_weight=class_weight)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Epoch 1/200
62393/62393 [==============================] - 40s 649us/step - loss: 0.1530 - binary_accuracy: 0.1722
Epoch 2/200
62393/62393 [==============================] - 39s 630us/step - loss: 0.1521 - binary_accuracy: 0.1681
Epoch 3/200
62393/62393 [==============================] - 39s 620us/step - loss: 0.1511 - binary_accuracy: 0.1742
Epoch 4/200
62393/62393 [==============================] - 39s 628us/step - loss: 0.1504 - binary_accuracy: 0.1979
Epoch 5/200
62393/62393 [==============================] - 38s 615us/step - loss: 0.1500 - binary_accuracy: 0.2140
Epoch 6/200
62393/62393 [==============================] - 39s 620us/step - loss: 0.1498 - binary_accuracy: 0.2023
Epoch 7/200
62393/62393 [==============================] - 38s 614us/step - loss: 0.1497 - binary_accuracy: 0.2176
Epoch 8/200
62393/62393 [==============================] - 38s 617us/step - loss: 0.1492 - binary_accuracy: 0.2397
Epoch 9/200
62393/62393 [==============================] - 39s 619us/step - loss

In [0]:
from sklearn.metrics import precision_score,accuracy_score
y_predicted=model.predict_classes(X_train)
y_predicted=y_predicted.reshape(1,-1)[0]
precision = precision_score(y_train, y_predicted)
accuracy=accuracy_score(y_train,y_predicted)
print("Train Precision is ",precision)
print("Train Accuracy is ",accuracy)
pd.crosstab(y_train, y_predicted, rownames=['True'], colnames=['Predicted'], margins=True)

Train Precision is  0.3349296230383433
Train Accuracy is  0.6685205071081692


Predicted,0,1,All
True,,,
0.0,31360,20554,51914
1.0,128,10351,10479
All,31488,30905,62393


In [0]:
from sklearn.metrics import classification_report
print(classification_report(y_train,y_predicted))

              precision    recall  f1-score   support

         0.0       1.00      0.60      0.75     51914
         1.0       0.33      0.99      0.50     10479

    accuracy                           0.67     62393
   macro avg       0.67      0.80      0.63     62393
weighted avg       0.88      0.67      0.71     62393



In [0]:
model=Sequential()
model.add(LSTM(units=50,return_sequences=True,input_shape=(X_train.shape[1],45)))
model.add(Dropout(0.2))

model.add(LSTM(units=50,return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=50,return_sequences=True))
model.add(Dropout(0.2))


model.add(LSTM(units=50))
model.add(Dropout(0.2))


model.add(Dense(units=1,activation='sigmoid'))

class_weight={0:0.10,
               1:0.90}
model.compile(loss='binary_crossentropy',
                  optimizer='adam',
              metrics=['binary_accuracy'])

In [0]:
model.fit(X_train, y_train, batch_size=500, nb_epoch=300,class_weight=class_weight)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Epoch 1/300
62393/62393 [==============================] - 46s 729us/step - loss: 0.1525 - binary_accuracy: 0.1709
Epoch 2/300
62393/62393 [==============================] - 42s 668us/step - loss: 0.1513 - binary_accuracy: 0.1738
Epoch 3/300
62393/62393 [==============================] - 43s 692us/step - loss: 0.1503 - binary_accuracy: 0.1929
Epoch 4/300
62393/62393 [==============================] - 45s 717us/step - loss: 0.1503 - binary_accuracy: 0.2011
Epoch 5/300
62393/62393 [==============================] - 43s 696us/step - loss: 0.1500 - binary_accuracy: 0.2078
Epoch 6/300
62393/62393 [==============================] - 44s 712us/step - loss: 0.1496 - binary_accuracy: 0.2189
Epoch 7/300
62393/62393 [==============================] - 44s 698us/step - loss: 0.1495 - binary_accuracy: 0.2146
Epoch 8/300
62393/62393 [==============================] - 43s 685us/step - loss: 0.1493 - binary_accuracy: 0.2289
Epoch 9/300
62393/62393 [==============================] - 42s 673us/step - loss

In [0]:
from sklearn.metrics import precision_score,accuracy_score
y_predicted=model.predict_classes(X_train)
y_predicted=y_predicted.reshape(1,-1)[0]
precision = precision_score(y_train, y_predicted)
accuracy=accuracy_score(y_train,y_predicted)
print("Train Precision is ",precision)
print("Train Accuracy is ",accuracy)


from sklearn.metrics import classification_report
print(classification_report(y_train,y_predicted))

Train Precision is  0.39436885865457294
Train Accuracy is  0.742455083102271
              precision    recall  f1-score   support

         0.0       1.00      0.69      0.82     51914
         1.0       0.39      1.00      0.56     10479

    accuracy                           0.74     62393
   macro avg       0.70      0.84      0.69     62393
weighted avg       0.90      0.74      0.77     62393



In [0]:
pd.crosstab(y_train, y_predicted, rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,0,1,All
True,,,
0.0,35889,16025,51914
1.0,44,10435,10479
All,35933,26460,62393


In [0]:
y_predicted=model.predict_classes(X_test)
y_predicted=y_predicted.reshape(1,-1)[0]
precision = precision_score(y_test, y_predicted)
accuracy=accuracy_score(y_test,y_predicted)
print("Test Precision is ",precision)
print("Test Accuracy is ",accuracy)

Test Precision is  0.18471337579617833
Test Accuracy is  0.5333333333333333


In [0]:
pd.crosstab(y_test, y_predicted, rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,0,1,All
True,,,
0.0,1681,1408,3089
1.0,342,319,661
All,2023,1727,3750


In [0]:
model=Sequential()
model.add(LSTM(units=50,return_sequences=True,input_shape=(X_train.shape[1],45)))
model.add(Dropout(0.2))

model.add(LSTM(units=50,return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=50,return_sequences=True))
model.add(Dropout(0.2))


model.add(LSTM(units=50))
model.add(Dropout(0.2))


model.add(Dense(units=1,activation='sigmoid'))

class_weight={0:0.10,
               1:0.90}
model.compile(loss='binary_crossentropy',
                  optimizer='adam',
              metrics=['binary_accuracy'])

In [0]:
model.fit(X_train, y_train, batch_size=500, nb_epoch=500,class_weight=class_weight)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Epoch 1/500
62393/62393 [==============================] - 42s 671us/step - loss: 0.1529 - binary_accuracy: 0.1685
Epoch 2/500
62393/62393 [==============================] - 40s 641us/step - loss: 0.1519 - binary_accuracy: 0.1681
Epoch 3/500
62393/62393 [==============================] - 40s 634us/step - loss: 0.1507 - binary_accuracy: 0.1818
Epoch 4/500
62393/62393 [==============================] - 39s 632us/step - loss: 0.1505 - binary_accuracy: 0.1913
Epoch 5/500
62393/62393 [==============================] - 40s 635us/step - loss: 0.1499 - binary_accuracy: 0.2066
Epoch 6/500
62393/62393 [==============================] - 40s 634us/step - loss: 0.1494 - binary_accuracy: 0.2147
Epoch 7/500
62393/62393 [==============================] - 40s 639us/step - loss: 0.1493 - binary_accuracy: 0.2259
Epoch 8/500
62393/62393 [==============================] - 40s 637us/step - loss: 0.1492 - binary_accuracy: 0.2223
Epoch 9/500
62393/62393 [==============================] - 39s 629us/step - loss

In [0]:
from sklearn.metrics import precision_score,accuracy_score
y_predicted=model.predict_classes(X_train)
y_predicted=y_predicted.reshape(1,-1)[0]
precision = precision_score(y_train, y_predicted)
accuracy=accuracy_score(y_train,y_predicted)
print("Train Precision is ",precision)
print("Train Accuracy is ",accuracy)

pd.crosstab(y_train, y_predicted, rownames=['True'], colnames=['Predicted'], margins=True)

Train Precision is  0.4927084808155175
Train Accuracy is  0.8270959883320245


Predicted,0,1,All
True,,,
0.0,41165,10749,51914
1.0,39,10440,10479
All,41204,21189,62393


In [0]:
y_predicted=model.predict_classes(X_test)
y_predicted=y_predicted.reshape(1,-1)[0]
precision = precision_score(y_test, y_predicted)
accuracy=accuracy_score(y_test,y_predicted)
print("Test Precision is ",precision)
print("Test Accuracy is ",accuracy)

pd.crosstab(y_test, y_predicted, rownames=['True'], colnames=['Predicted'], margins=True)

Test Precision is  0.19191226867717615
Test Accuracy is  0.584


Predicted,0,1,All
True,,,
0.0,1910,1179,3089
1.0,381,280,661
All,2291,1459,3750
